##ConversationSummaryBufferMemory
有两个思路。一个是保存以前的交互。
另一个是：以前是旧交互信息会完全覆盖，现在是把旧信息和现在的信息汇总。
而且，用token的长度来限制内容的长度，而不是用交互的次数。

In [ ]:
import os
print(os.environ["OPENAI_API_KEY"])

In [68]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationEntityMemory
from langchain.chat_models import ChatOpenAI
import os
#os.environ["OPENAI_API_KEY"] = "sk-写在.env中" 

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo"
    )



In [9]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "嘿！"}, {"output": "怎么了"})
memory.save_context({"input": "你没事吧"}, {"output": "我很好，没事"})



In [10]:
memory.load_memory_variables({})

{'history': "System: The human greets the AI in Chinese, and the AI responds by asking what's up. The human asks if the AI is okay in Chinese, and the AI responds in Chinese that it is doing well and nothing is wrong."}

In [11]:
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'There is currently no information provided to form a summary.'

使用能自动汇总的memery。
好像有问题，自动汇总其实是自动向官方发汇总请求，会造成回复混乱。而且，默认模板并不能保证自己的重点在汇总中保存下来，所以要自己重新设计prompt模板才行。


In [12]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage

llm=ChatOpenAI()
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=40),
    verbose=True,
)

conversation_with_summary.predict(input="嘿, 你那边怎么样?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 嘿, 你那边怎么样?
AI:

> Finished chain.


'你好! 我的运行状况很好，谢谢关心。我当前正以每秒一百万次的速度处理数据。您需要我为您做些什么吗？'

In [14]:
conversation_with_summary.predict(input="帮我写一份文案吧!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks how it is doing. The AI responds that its performance is good and it is currently processing data at one million times per second. The AI also asks the human if it can assist them with anything.
Human: 帮我写一份文案吧!
AI:

> Finished chain.


'当然可以帮你写文案。您需要什么类型的文案？比如说广告、产品描述或是一篇文章等等。请告诉我详细的要求，我会根据您的需求进行创作。'

In [16]:
conversation_with_summary.predict(input="写一篇关于AI发展趋势的文章，题目：《未来50年AI会如何发展》")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks about its performance, which is good and processing data rapidly. The AI offers to assist with any task, and the human requests help with writing an article about the future of AI. The AI agrees to research the latest trends, developments, challenges, and solutions in the field and use reliable sources to ensure accuracy. The AI expresses gratitude for the trust and promises to do its best to complete the article.
Human: 写一篇关于AI发展趋势的文章，题目：《未来50年AI会如何发展》
AI:

> Finished chain.


'非常好的题目！根据我的研究，未来50年AI将继续迅速发展。首先，AI技术将变得更加普及，因为越来越多的企业和组织开始认识到AI的价值，尤其是在自动化、数据分析和智能沟通方面。其次，AI系统将变得更加智能化，因为它们将学会更多的人类语言和行为，以及对环境和情境的敏感程度，从而更好地服务人类社会。\n\n但是，AI的发展也可能带来一些挑战。例如，随着AI技术的发展，社会将不可避免地面临某些职业的失业风险。因此，我们需要采取有效的政策和措施来解决这些问题，从而确保AI技术的全面性和可持续性。此外，我们需要加强AI系统的安全和隐私，以保护用户的数据和信息不被非法使用或盗窃。\n\n总的来说，未来50年AI将持续发展，并为人类社会带来更多的好处和机会。让我们一起期待这样一个更加智慧、先进和美好的未来！'

## 用vector_store建立一个库。看还会不会忘记。还能与知识库联系起来。


In [ ]:
import streamlit as st
from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
import tempfile

In [ ]:
user_api_key = st.sidebar.text_input(
    label="#### Your OpenAI API key  ",
    placeholder="Paste your openAI API key, sk-",
    type="password")

uploaded_file = st.sidebar.file_uploader("upload", type="csv")

In [ ]:
if uploaded_file :
   #use tempfile because CSVLoader only accepts a file_path
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name

    loader = CSVLoader(file_path=tmp_file_path, encoding="utf-8")
    data = loader.load() 

In [ ]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embeddings)



我们通过为其提供所需的聊天模型 gpt-3.5-turbo（或 gpt-4）和 FAISS vectorstore 来添加 ConversationalRetrievalChain，
FAISS vectorstore 存储我们通过 OpenAIEmbeddings() 转换为向量的文件。
Chain 允许我们拥有一个有记忆的聊天机器人，同时依靠它vectorstore从我们的文档中找到相关信息。

此功能允许我们向 ConversationalRetrievalChain 提供用户的问题和对话历史记录以生成聊天机器人的响应。
st.session_state['history']存储用户在 Streamlit 网站上的对话历史记录。

In [ ]:
chain = ConversationalRetrievalChain.from_llm(
    llm = ChatOpenAI(temperature=0.0,model_name='gpt-3.5-turbo'),
    retriever=vectorstore.as_retriever())


In [ ]:
def conversational_chat(query):
        
    result = chain({"question": query, 
        "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
        
    return result["answer"]

创建 st.session_state['history'] 和聊天中显示的第一条消息来初始化聊天机器人会话。
['generated']对应于聊天机器人的响应。
['past']对应于用户提供的消息。
容器不是必需的，但可以通过将用户的问题区域放置在聊天消息下方来帮助改进 UI。

In [ ]:
if 'history' not in st.session_state:
        st.session_state['history'] = []

if 'generated' not in st.session_state:
        st.session_state['generated'] = ["Hello ! 可以问我任何关于《 " + uploaded_file.name + " 》的问题 "]

if 'past' not in st.session_state:
        st.session_state['past'] = ["Hey !  "]
        
    #container for the chat history
response_container = st.container()
    #container for the user's text input
container = st.container()

In [ ]:
with container:
    with st.form(key='my_form', clear_on_submit=True):
            
        user_input = st.text_input("Query:", placeholder="Talk about your csv data here (:", key='input')
        submit_button = st.form_submit_button(label='Send')
            
    if submit_button and user_input:
        output = conversational_chat(user_input)
            
        st.session_state['past'].append(user_input)
        st.session_state['generated'].append(output)

    if st.session_state['generated']:
        with response_container:
            for i in range(len(st.session_state['generated'])):
                message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="big-smile")
                message(st.session_state["generated"][i], key=str(i), avatar_style="thumbs")

剩下的就是启动脚本：

streamlit run name_of_your_chatbot.py 
#run 用你的文件名

## 访问外部网站获得信息

In [17]:
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain


from langchain.prompts import PromptTemplate

template = """在 >>> 和 <<< 之间，是需要从百度搜索的信息。
找到大括号 '{query}' 中请求的答案，找不到就说"没有找到" .
用如下公式
Extracted:<回答 or "没有找到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [29]:
chain = LLMRequestsChain(llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))

question = "什么是 Deep Lake ?"
inputs = {
    "query": question,
    "url": "https://www.baidu.com/s?ie=utf-8&wd=" + question.replace(" ", "+")
}

chain(inputs)

{'query': '什么是 Deep Lake ?',
 'url': 'https://www.baidu.com/s?ie=utf-8&wd=什么是+Deep+Lake+?',
 'output': 'Deep Lake是一个位于加拿大安大略省的深水湖，它位于多伦多以西约100公里处，是一个深水湖，湖水深度可达100米。'}

In [ ]:
question = "这个网页在讲什么https://python.langchain.com/en/latest/use_cases/question_answering/semantic-search-over-chat.html?"
inputs = {
    "query": question,
    "url": "https://www.baidu.com/s?ie=utf-8&wd=" + question.replace(" ", "+")
}

chain(inputs)

In [21]:
from langchain.agents import Tool, AgentExecutor, Agent
from langchain import OpenAI, SerpAPIWrapper

#search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=chain,
        description="useful for when you need to answer questions about current events",
        return_direct=True
    )
]

In [55]:
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain


from langchain.prompts import PromptTemplate

template_url = """你是内容分析员，你的工作是解析出human关心的网页地址.你不需要回答问话，只需要解析出网页地址。
只回答有效的url地址，不加任何修饰
例如：
  问: 这个网页讲什么？https://wwww.xxaa.com/bbcc.html
  答: https://wwww.xxaa.com/bbcc.html 

以下是human问话:
 {query}

"""

URL_PROMPT = PromptTemplate(
    input_variables=["query"],
    template=template_url,
)

template_content = """在 >>> 和 <<< 之间,是需要从网页地址url获取的内容,
获取大括号 '{url}' 中网页的内容，获取不到就说"地址可能有误" .
用如下公式
Extracted:<回答 or "没有找到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["url", "requests_result"],
    template=template_content,
)

In [58]:
llm = OpenAI(temperature=0)
url_chain = LLMChain(llm=llm, prompt=URL_PROMPT,verbose=True,output_key="url")
question = "这个网页在讲什么?https://python.langchain.com/en/latest/use_cases/question_answering/semantic-search-over-chat.html"
inputs_url = {
    "query": question,
    
   # "url": "https://www.baidu.com/s?ie=utf-8&wd=" + question.replace(" ", "+")
}
url_chain(inputs_url)



> Entering new LLMChain chain...
Prompt after formatting:
你是内容分析员，你的工作是解析出human关心的网页地址.你不需要回答问话，只需要解析出网页地址。
只回答有效的url地址，不加任何修饰
例如：
  问: 这个网页讲什么？https://wwww.xxaa.com/bbcc.html
  答: https://wwww.xxaa.com/bbcc.html
解析出的结果命名为url  

以下是human问话:
 这个网页在讲什么?https://python.langchain.com/en/latest/use_cases/question_answering/semantic-search-over-chat.html



> Finished chain.


{'query': '这个网页在讲什么?https://python.langchain.com/en/latest/use_cases/question_answering/semantic-search-over-chat.html',
 'url': '\nURL: https://python.langchain.com/en/latest/use_cases/question_answering/semantic-search-over-chat.html'}

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = LLMRequestsChain(llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))

question = "这个网页在讲什么https://python.langchain.com/en/latest/use_cases/question_answering/semantic-search-over-chat.html?"

inputs = {
    "query": question,
    #"url":url_chain(inputs),
   # "url": "https://www.baidu.com/s?ie=utf-8&wd=" + question.replace(" ", "+")
}

#chain(inputs)
overall_chain = SimpleSequentialChain(chains=[url_chain, chain], verbose=True)
overall_chain.run(input=inputs)